# EMMO Browser example
This notebook is an example on how to utilize triples to generate an ontology documentation browser

## Installation
    $ pip install ipytree
    
    If you use JupyterLab<=2:
    $ jupyter labextension install @jupyter-widgets/jupyterlab-manager ipytree

    If you have notebook 5.2 or below, you also need to execute:
    $ jupyter nbextension enable --py --sys-prefix ipytree

In [1]:
from typing import Tuple, List, Dict
from dataclasses import dataclass

# Our humble triplestore
triples: List[Tuple[str, str, str]] = []

# Helper functions for perfoming triple matching
create_match = lambda tr: lambda s, p, o: [t for t in tr if (not s or t[0] == s) and (not p or t[1] == p) and (not o or t[2] == o)]
match = lambda s = None, p = None, o = None: create_match(triples)(s, p, o)
match_first = lambda s = None, p = None, o = None : next(iter(match(s, p, o) or []), (None, None, None))


In [2]:
# Import triples onto the triplestore
import shlex 
    

filenames = ['battery.nt',
             'crystallography.nt',
             'emmo.nt',
             'properties.nt',
             'characterisation.nt',
             'electrochemistry.nt',
             'physmet.nt']
for filename in filenames:
    with open(filename, 'r') as file:            
        lines = shlex.split(file.read());        
        for i in range(0, len(lines), 4):
            triples.append((lines[i], lines[i+1], lines[i+2])) 
          

In [3]:
len(triples)

7914

In [4]:
from ipytree import Tree, Node
from ipywidgets import HBox, VBox, HTML, Text, Output, Layout, Combobox
from typing import Any
from jinja2 import Template
import html
from namespaces import RDFS, SKOS, EMMO

# Read the jinja2-template from file
with open("card.html.j2", "r") as f:
    template = f.read()
    jtemplate = Template(template)

# Declare the output window for the concept documentation
documentation = Output(layout=Layout(min_width='50%'))
o = Output()

    
# Helper-map that maps ipytree node-uuids to concept-URI
nodemap = {}
search_index = {}
phrases = []
combo_search: Combobox = Combobox(options=[], ensure_option=True, enabled=False)

def on_value_change(change):
    """ Select (exact) matching labels """
    word = change['new'].lower()

    if word in search_index:        
        search_index[word].selected = True        
    else:
        opt = []
        combo_search.value = ''
        for phrase in phrases:
            if word in phrase.lower():
                opt.append(phrase)
        combo_search.options = opt

def on_combo_change(change):
    """ Value was selected from the combo box"""
    word = change['new'].lower()
    if word in search_index:
        search_index[word].selected = True

search = Text(placeholder='Enter search string here', description='Search:')
search.observe(on_value_change, names='value')
combo_search.observe(on_combo_change, names='value')

def match_to_string(concept, predicate):
    """ Concatenate and wash matching object strings """
    return "".join([(html.unescape(text))
                    .replace('@en', '\n')                    
                    .replace('\\n', '<br/>\n') 
                    for (_,_,text) in match(concept, predicate)])

def handle_click(event):
    """ Event to be called when a node is clicked """
    concept = nodemap[event['owner']._id]
    documentation.clear_output()
    with documentation:        
        display(HTML(value=jtemplate.render(
            label=event['owner'].name, 
            concept=html.escape(concept), 
            elucidation=match_to_string(concept, EMMO.elucidation),
            comments=match_to_string(concept, RDFS.comment), 
            etymology=match_to_string(concept, EMMO.etymology),
            subclass=match_to_string(concept, RDFS.subClassOf),
        )))
        
        
def generate_node_tree(root):
    """ Recursively generate the ipytree nodes """
    sub_nodes = []
    
    # Recursively generate and append sub-nodes
    for (sub_node,_,_) in match(None, RDFS.subClassOf, root):        
        sub_nodes.append(generate_node_tree(sub_node))
        
    # Find the prefLabel (node name)
    try:
        (_,_,lbl) = match_first(root, SKOS.prefLabel, None)
        lbl = lbl.replace('@en','')
    except:
        print (f'WARNING! {root} has not label!')
        lbl = "MISSING"
        
    if sub_nodes:
        node = Node(lbl, sub_nodes, opened=False, icon_style="success")
    else:
        node = Node(lbl, sub_nodes, opened=False, icon="leaf", icon_style="success")
        
    # Add observer to handle interactions
    node.observe(handle_click, 'selected')
    nodemap[node._id] = root
    search_index[lbl.lower()] = node
    if not lbl in phrases:
        phrases.append(lbl) 
        
    return node

# Generate the node tree
emmo_node = generate_node_tree(EMMO.Emmo)

WARNING! <http://emmo.info/physmet#EMMO_a806eb67_5360_45ca_9b58_3ab259d69ca9> has not label!
WARNING! <http://emmo.info/domain-crystallography/cif_top#CIF> has not label!


In [5]:
tree=Tree(layout=Layout(min_width='50%'))
tree.add_node(emmo_node)

upper = HBox(children=[search, combo_search], layout=Layout(align_items='stretch', width='100%'))
box = HBox(children=[tree, documentation], layout=Layout(align_items='stretch', width='100%'))
vbox = VBox(children=[upper, box])
display (vbox)
display (o)

Output()